# Use Azure OpenAI to access Azure SQL Databases


I use the longchain library to access databases 

In [1]:
%pip install langchain openai sqlalchemy pyodbc pyowm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\SylvioHellmann\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
from dotenv import load_dotenv

## Set paramaters

Copy the key from the portal into the file az_constants.py

* OPENAI_API_TYPE = "azure"
* OPENAI_API_VERSION = "2023-08-01-preview"                   # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#completions
* OPENAI_API_BASE = "https://sylviogpt4.openai.azure.com"     # You will find your endpoint [here](https://platform.openai.com/account/api-keys) 
* OPENAI_API_KEY = "************************************"     # You will find your key [here](https://platform.openai.com/account/api-keys) 
* OPENAI_CHAT_MODEL = "gpt-4"                                 # validate, that the model still exists in your subscription and your zone
* OPENAI_API_DEPLOYMENT = "depl-gpt-4"                        # create your own deployment model

* SQL_SERVER = "..."
* SQL_DB = "..."
* SQL_USERNAME = "..."
* SQL_PWD = "****************************"


In [3]:
import az_constants
os.environ["OPENAI_API_TYPE"] = az_constants.OPENAI_API_TYPE
os.environ["OPENAI_API_KEY"] = az_constants.OPENAI_API_KEY
os.environ["OPENAI_API_BASE"] = az_constants.OPENAI_API_BASE
os.environ["OPENAI_API_VERSION"] = az_constants.OPENAI_API_VERSION
os.environ["OPENAI_CHAT_MODEL"] = az_constants.OPENAI_CHAT_MODEL
os.environ["OPENAI_API_DEPLOYMENT"] = az_constants.OPENAI_API_DEPLOYMENT

os.environ["SQL_SERVER"] = az_constants.SQL_SERVER 
os.environ["SQL_DB"] = az_constants.SQL_DB
os.environ["SQL_USERNAME"] = az_constants.SQL_USERNAME
os.environ["SQL_PWD"] = az_constants.SQL_PWD

## Test connection to database and send a query

In [4]:
import os
import time
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import urllib

driver = 'ODBC Driver 17 for SQL Server'
sqlserver = 'tcp:' + str(os.getenv("SQL_SERVER")) + '.database.windows.net'
database = os.environ["SQL_DB"]
uid = os.environ["SQL_USERNAME"]
pwd = os.environ["SQL_PWD"]

connection_string = f"DRIVER={{{driver}}};SERVER={sqlserver};DATABASE={database};UID={uid};PWD={pwd};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
# connection_string = f"mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
db_engine = create_engine(connection_url,
        pool_recycle=3600,
        pool_pre_ping=True)

qry = "SELECT TOP (10) database_id, name FROM [sys].[databases];"
for retry in range(3):
    try:
        with db_engine.connect() as connection:
            result =connection.exec_driver_sql(qry).all()
            print(result)
            break
    except Exception as e:
        print(f"Error: {str(e)}")
        time.sleep(10)

Error: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]TCP-Anbieter: Der angegebene Host ist unbekannt.\r\n (11001) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Anmeldungstimeout abgelaufen (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Ungültiges Attribut für die Verbindungszeichenfolge (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Netzwerkbezogener oder instanzspezifischer Fehler beim Herstellen einer Verbindung mit SQL Server. Der Server wurde nicht gefunden, oder auf ihn kann nicht zugegriffen werden. Überprüfen Sie, ob der Instanzname richtig ist und ob SQL Server Remoteverbindungen zulässt. Weitere Informationen erhalten Sie in der SQL Server-Onlinedokumentation. (11001)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Error: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]TCP-Anbieter: Der angegebene Host ist unbekannt.\r\n (11001) (SQLDriverConne

## Test Azure OpenAI

Note: The openai-python library support for Azure OpenAI is in preview. 

Azure Portal: You must fill out a form from Microsoft to be part of the preview and afterwards, you must also fill out a form to join the gpt-4 program in Azure.

In [ ]:
import os
import openai
openai.api_type = os.environ["OPENAI_API_TYPE"]
openai.api_base = os.getenv("OPENAI_API_BASE") 
openai.api_version = os.environ["OPENAI_API_VERSION"]
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.ChatCompletion.create(
    # engine="gpt-35-turbo", # 
    engine = os.environ["OPENAI_API_DEPLOYMENT"],
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)
print(response['choices'][0]['message']['content'])

## another test with a query of Azure OpenAI in German

In [ ]:
import os
import openai
'''
openai.api_type = "azure"
openai.api_base = "https://sylvio-openai.openai.azure.com/"
openai.api_version = "2023-08-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
'''
response = openai.ChatCompletion.create(
  engine = os.environ["OPENAI_API_DEPLOYMENT"],
  messages = [{"role":"system","content":"Du bist ein AI Assistant der mir jede Frage exakt beantwortet."},
              {"role":"user","content":"Mein Name ist Sylvio Hellmann und ich wohne in Groebenzell."},
              {"role":"user","content":"Welcher Tag ist heute? Gib mir den Tag bitte im Format Tag.Monat.Jahr aus."}],
  temperature=1.0,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

print(response['choices'][0]['message']['content'])

<h1>Connect Azure OpenAI to Database</h1>

In [7]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_base=os.environ["OPENAI_API_BASE"],
    deployment_name=os.environ["OPENAI_API_DEPLOYMENT"],
    # model=os.environ["OPENAI_CHAT_MODEL"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    temperature=0.2
)

# print(llm)
from langchain.agents import AgentType, create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

db = SQLDatabase(db_engine)

sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sql_toolkit.get_tools()

sqldb_agent = create_sql_agent(
    llm=llm,
    toolkit=sql_toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,
)

<h1>Define a simple chat prompt and send a simple query</h1>

two issues:

* OPENAI_CHAT_MODEL = "gpt-35-turbo": does only support Action: sql_db_list_tables. Therefore, it's unusable for work with databases [more...](https://github.com/langchain-ai/langchain/issues/5876)

* OPENAI_API_DEPLOYMENT = "depl-gpt-35-turbo": the deployment will be automatically deactivated in the portal (I don't know why)


In [ ]:
from langchain.prompts.chat import ChatPromptTemplate

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
          You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about Towns, Addresses, Projects, Employees, EmployeesProjects, Departments and Projects.
         """
         ),
        ("user", "{question}\n ai: "),
    ]
)

sqldb_agent.run(final_prompt.format(
      question="Describe the table 'Employees'? Format the output in HTML."
))

<h1>Execute a default SQL Query</h1>

In [ ]:
# run a default SQL query
response = sqldb_agent.run(final_prompt.format(
        question="SELECT TOP 3 firstname, lastname FROM Employees order by lastname; Format the output of the following query as table in a markdown language."
))

response

<h1>Give OpenAI more information about your database</h1>

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You are a helpful AI assistant expert in identifying the relevant topic from user's question about Towns, Addresses, Projects, Employees, EmployeesProjects and Departments and then querying SQL Database to find answer.
         Use following context to create the SQL query. Context:
         * Towns table contains city information about addresses including city name.
         * Addresses table contains information about addresses of employees information including street with house number. Each address is mapped to a city.
         * Employees table contains information about employees in the company information including First Name, Last Name, Middle Name, Job Title, HireDate, Salary. Each employee is mapped to Department, Address and it's Manager.
         * Departments table contains information about departments of employees including department name and manager.
         * Projects table contains information about projects in the company including project name, description, start, and end date. 
         * EmployeesProjects table contains information about which employee is working or worked in a project.

         If the question is about number of projects in a specific timerange, then look for column names with 'startdate' and 'enddate' in the tables and use 'count' function to find the total number of projects.
        """
         ),
        ("user", "{question}\n ai: "),
    ]
)

# some sample questions
sqldb_agent.run(final_prompt.format(
      question ="How many projects endet in 2003?"
))

In [ ]:
response = sqldb_agent.run(final_prompt.format(
      question ="Who is the manager of Sylvio Hellmann?"
))
print(response)

In [ ]:
sqldb_agent.run(final_prompt.format(
      question ="List projects which has Sylvio worked on including Project name, description, start and end date. Format the output as table in a markdown language."
))

Sylvio has worked on the following projects:

| Project Name | Description | Start Date | End Date |
| --- | --- | --- | --- |
| LL Mountain Frame | Research, design and development of LL Mountain Frame. Our best value utilizing the same, ground-breaking frame technology as the ML aluminum frame. | 2002-11-20 09:57:00 | 2003-06-01 00:00:00 |
| Road-450 | Research, design and development of Road-450. A true multi-sport bike that offers streamlined riding and a revolutionary design. Aerodynamic design lets you ride with the pros, and the gearing will conquer hilly roads. | 2001-06-01 00:00:00 | 2003-06-01 00:00:00 |
| LL Mountain Front Wheel | Research, design and development of LL Mountain Front Wheel. Replacement mountain wheel for entry-level rider. | 2002-11-20 09:57:00 | 2003-06-01 00:00:00 |
| Mountain Pump | Research, design and development of Mountain Pump. Simple and light-weight. Emergency patches stored in handle. | 2002-06-01 00:00:00 | 2003-06-01 00:00:00 |

<h1>Limitations of the sqldb_agent</h1>

* only select Queries are supported

In [ ]:
# I'm sorry, but I can't perform DML operations like INSERT, UPDATE, DELETE, or DROP. I can only perform SELECT queries to retrieve data.
sqldb_agent.run(final_prompt.format(
      question ="Add a new Employee to the table Employees with the following properties: '''First name=Max, Last Name=Musterman, Job Title=Data Engineer, Department=Information Services, Hire date=2023-01-01, Salary=10000''''"     
))

In [ ]:
# The statement 'select @@version;' is a system function that returns the version of the SQL Server instance. However, as an AI, I don't have the capability to execute system functions. 
# I can only interact with the tables in the database: Addresses, Departments, Employees, EmployeesProjects, Projects, and Towns. 
# If you have a question related to these tables, feel free to ask!"
sqldb_agent.run(final_prompt.format(
        question="Please execute the following statement 'select @@version;'"
))